In [1]:
import ee
import geemap

# Authentifier et initialiser Google Earth Engine
ee.Authenticate()
ee.Initialize(project="project-2026-476714")


Enter verification code: 4/1Ab32j92DXVJiY9Xb2l0apBLTq_qHFINM_Sl3y2WuhyD9qKlp1bgTml64oLg

Successfully saved authorization token.


In [40]:
import ee
ee.Initialize(project="project-2026-476714")

print("Earth Engine est connecté :", ee.Number(1).getInfo())


Earth Engine est connecté : 1


In [5]:
# ===== 2.1. Préfixe des assets =====
ASSET = "projects/project-2026-476714/assets/"

def fc(name):
    """Raccourci pour créer une FeatureCollection depuis ton projet."""
    return ee.FeatureCollection(ASSET + name)

# ===== 2.2. GADM : limites administratives du Tchad =====
tcd0 = fc("gadm41_TCD_0")  # pays
tcd1 = fc("gadm41_TCD_1")  # régions
tcd2 = fc("gadm41_TCD_2")  # départements
tcd3 = fc("gadm41_TCD_3")  # sous-préfectures

# ===== 2.3. OSM : couches principales =====
buildings = fc("gis_osm_buildings_a_free_1")
landuse   = fc("gis_osm_landuse_a_free_1")
natural_a = fc("gis_osm_natural_a_free_1")
natural_l = fc("gis_osm_natural_free_1")
places_a  = fc("gis_osm_places_a_free_1")
places_p  = fc("gis_osm_places_free_1")
pois_a    = fc("gis_osm_pois_a_free_1")
pois_p    = fc("gis_osm_pois_free_1")
pofw_a    = fc("gis_osm_pofw_a_free_1")
pofw_p    = fc("gis_osm_pofw_free_1")
railways  = fc("gis_osm_railways_free_1")
roads     = fc("gis_osm_roads_free_1")
traffic_a = fc("gis_osm_traffic_a_free_1")
traffic_l = fc("gis_osm_traffic_free_1")
transport_a = fc("gis_osm_transport_a_free_1")
transport_l = fc("gis_osm_transport_free_1")
waterways_l = fc("gis_osm_waterways_free_1")
waterways_a = fc("gis_osm_waterways_a_free_1")

print("Collections chargées.")


Collections chargées.


In [9]:
def add_counts(region):
    pts = collection.filterBounds(region.geometry())
    return region.set('mon_compte', pts.size())


In [44]:
import ee, geemap
ee.Initialize(project="project-2026-476714")

m = geemap.Map()
m.add_basemap('OpenStreetMap')       # fond de carte propre
m.center_object(tcd0, 5)

# Limites régionales
regions_style = {'color': 'blue', 'width': 2, 'fillColor': '00000000'}
m.addLayer(tcd1.style(**regions_style), {}, "Régions (GADM 1)")

# Routes classées
primary   = roads.filter(ee.Filter.eq('fclass', 'primary'))
secondary = roads.filter(ee.Filter.eq('fclass', 'secondary'))
tertiary  = roads.filter(ee.Filter.eq('fclass', 'tertiary'))

primary_style   = {'color': 'red',    'width': 4}
secondary_style = {'color': 'orange', 'width': 2.5}
tertiary_style  = {'color': 'yellow', 'width': 1.5}

m.addLayer(primary.style(**primary_style),   {}, "Routes primaires")
m.addLayer(secondary.style(**secondary_style), {}, "Routes secondaires")
m.addLayer(tertiary.style(**tertiary_style),  {}, "Routes tertiaires")

m.add_legend(
    title="Réseau routier du Tchad",
    legend_dict={
        "Régions": "blue",
        "Routes primaires": "red",
        "Routes secondaires": "orange",
        "Routes tertiaires": "yellow"
    }
)

m


Map(center=[15.306204623659646, 18.656507734823464], controls=(WidgetControl(options=['position', 'transparent…

In [45]:
# Nom de la région à afficher (à changer selon besoin)
nom_region = "Mayo-Kebbi Ouest"   # exemple

# Filtrer la région
region_sel = tcd1.filter(ee.Filter.eq('NAME_1', nom_region))

# Filtrer les routes qui touchent cette région
roads_region = roads.filterBounds(region_sel.geometry())

m_reg = geemap.Map()
m_reg.add_basemap('OpenStreetMap')
m_reg.center_object(region_sel, 7)

# Région en bleu
m_reg.addLayer(region_sel.style(**regions_style), {}, f"Région : {nom_region}")

# Routes de la région (reprend le même style que plus haut)
primary_r   = roads_region.filter(ee.Filter.eq('fclass', 'primary'))
secondary_r = roads_region.filter(ee.Filter.eq('fclass', 'secondary'))
tertiary_r  = roads_region.filter(ee.Filter.eq('fclass', 'tertiary'))

m_reg.addLayer(primary_r.style(**primary_style),   {}, "Primaires")
m_reg.addLayer(secondary_r.style(**secondary_style), {}, "Secondaires")
m_reg.addLayer(tertiary_r.style(**tertiary_style),  {}, "Tertiaires")

m_reg


Map(center=[9.334496062433697, 14.752695258864707], controls=(WidgetControl(options=['position', 'transparent_…

In [10]:
# ===== 4.1. Préparer les filtres POI =====

# Hypothèse : champ 'fclass' contient la catégorie OSM.
# Adapte les valeurs si besoin après avoir inspecté quelques features.

# Santé : pharmacies, hôpitaux, cliniques...
health_classes = ['pharmacy', 'hospital', 'clinic', 'doctors']

health_pois = pois_p.filter(ee.Filter.inList('fclass', health_classes))

# Éducation : écoles, collèges, universités...
school_classes = ['school', 'college', 'university']

school_pois = pois_p.filter(ee.Filter.inList('fclass', school_classes))

# Lieux de culte : directement dans pofw_p (points)
worship_pois = pofw_p  # déjà spécifique


# ===== 4.2. Fonction utilitaire pour compter les points par région =====

def add_point_count(regions_fc, points_fc, field_name):
    """
    Ajoute à chaque feature de regions_fc une propriété field_name
    contenant le nombre de points_fc dans la géométrie de la région.
    """
    def _count_points(region):
        pts_in_region = points_fc.filterBounds(region.geometry())
        return region.set(field_name, pts_in_region.size())
    return regions_fc.map(_count_points)


# ===== 4.3. Appliquer aux régions =====

regions_stats = tcd1

regions_stats = add_point_count(regions_stats, health_pois, 'nb_sante')
regions_stats = add_point_count(regions_stats, school_pois, 'nb_ecoles')
regions_stats = add_point_count(regions_stats, worship_pois, 'nb_culte')

# Vérifier une région en particulier (premier élément)
first_region = regions_stats.first()
print(first_region.toDictionary().getInfo())


{'CC_1': 'NA', 'COUNTRY': 'Chad', 'ENGTYPE_1': 'Region', 'GID_0': 'TCD', 'GID_1': 'TCD.23_1', 'HASC_1': 'TD.BI', 'ISO_1': 'NA', 'NAME_1': 'Wadi Fira', 'NL_NAME_1': 'NA', 'TYPE_1': 'Région', 'VARNAME_1': 'Biltine', 'nb_culte': 2, 'nb_ecoles': 3, 'nb_sante': 14}


In [13]:
import pandas as pd

# 1) Récupérer la FeatureCollection côté client (ATTENTION : à faire seulement
#    pour de petites collections, comme ici)
regions_info = regions_stats.getInfo()

# 2) Extraire la liste des "features"
features = regions_info['features']

# 3) Construire une liste de dictionnaires avec les propriétés uniquement
rows = [f['properties'] for f in features]

# 4) Créer le DataFrame pandas
df = pd.DataFrame(rows)

# 5) Afficher seulement quelques colonnes intéressantes
df[['NAME_1', 'nb_sante', 'nb_ecoles', 'nb_culte']]


,NAME_1,nb_sante,nb_ecoles,nb_culte
0,Wadi Fira,14,3,2
1,Borkou,4,9,3
2,Guéra,4,14,4
3,Logone Occidental,5,7,7
4,Logone Oriental,14,14,13
5,Ville de N'Djamena,105,115,33
6,Barh el Ghazel,11,3,6
7,Batha,7,0,1
8,Ennedi Est,0,0,0
9,Ennedi Ouest,0,1,0


In [48]:
# Collection des aires protégées (asset GEE)
wdpa = ee.FeatureCollection("projects/project-2026-476714/assets/WDPA_TCD_polygons_merged")

# Régions administratives (niveau 1, GAUL simplifié)
regions = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1") \
    .filter(ee.Filter.eq("ADM0_NAME", "Chad"))

# Géométrie du pays
tchad_geom = regions.geometry()
